# 1. model 생성을 위한 units = 0이 아닌 데이터만 뽑아서 만든다

## 1-1 전체 데이터 load

In [1]:
from jw_package import *
from filling_missing import *
from functools import *

import configuration completed !
train + key + weather merging started
train + key + weather merging finished
function configuration completed ! 
Good to go !


In [2]:
each_station = []

for station_nbr in range(1,21):
    file_path = '../data/weather_best_close_final/{:02}.csv'.format(station_nbr)
    station = pd.read_csv(file_path,index_col=0)
    station.dropna(axis=0,how='any',inplace=True)
    each_station.append(station)

weather = reduce((lambda x,y : x.append(y)),each_station)

In [3]:
each_station = []

for station_nbr in range(1,21):
    file_path = '../data/weather_best_close_final/{:02}.csv'.format(station_nbr)
    station = pd.read_csv(file_path,index_col=0)
    station.dropna(axis=0,how='any',inplace=True)
    each_station.append(station)

weather = reduce((lambda x,y : x.append(y)),each_station)

In [4]:
sales = pd.read_csv('../data/basic/train.csv')
keys = pd.read_csv('../data/basic/key.csv')

In [5]:
data = sales.merge(keys).merge(weather)

In [6]:
data.head()

,date,store_nbr,item_nbr,units,station_nbr,tmax,tmin,tavg,depart,dewpoint,...,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day,codesum
0,2012-01-01,1,1,0,1,52.0,31.0,42.0,10.0,36.0,...,0.05,29.78,29.92,3.6,20.0,4.6,2012,1,1,RA FZFG BR
1,2012-01-01,1,2,0,1,52.0,31.0,42.0,10.0,36.0,...,0.05,29.78,29.92,3.6,20.0,4.6,2012,1,1,RA FZFG BR
2,2012-01-01,1,3,0,1,52.0,31.0,42.0,10.0,36.0,...,0.05,29.78,29.92,3.6,20.0,4.6,2012,1,1,RA FZFG BR
3,2012-01-01,1,4,0,1,52.0,31.0,42.0,10.0,36.0,...,0.05,29.78,29.92,3.6,20.0,4.6,2012,1,1,RA FZFG BR
4,2012-01-01,1,5,0,1,52.0,31.0,42.0,10.0,36.0,...,0.05,29.78,29.92,3.6,20.0,4.6,2012,1,1,RA FZFG BR


## 1-2 tmp 데이터로 copy후 units = 0이 아닌 것만 뽑아서 model 생성

In [30]:
tmp = data.copy()

In [31]:
tmp = tmp[tmp['units']!=0]

In [32]:
etc = ['day','month','year','station_nbr']
category = ['sunrise','sunset','codesum']

for each in etc+category:
    tmp.drop(each,axis=1,inplace=True)

In [33]:
del tmp['tavg']
del tmp['sealevel']
del tmp['wetbulb']
del tmp['stnpressure']
del tmp['tmin']
del tmp['tmax']
del tmp['dewpoint']
del tmp['avgspeed']
del tmp['resultspeed']

In [34]:
tmp.reset_index(drop = True, inplace = True)

In [35]:
tmp.tail()

,date,store_nbr,item_nbr,units,depart,heat,cool,snowfall,preciptotal,resultdir
118659,2014-10-29,35,66,14,6.0,13.0,0.0,0.0,0.02,23.0
118660,2014-10-30,35,16,16,6.0,13.0,0.0,0.0,0.02,30.0
118661,2014-10-30,35,66,6,6.0,13.0,0.0,0.0,0.02,30.0
118662,2014-10-31,35,16,18,2.0,24.0,0.0,0.0,0.00,5.0
118663,2014-10-31,35,66,10,2.0,24.0,0.0,0.0,0.00,5.0


# 2. 이부분은 codesum_v1.csv 데이터 load

In [36]:
codesum = pd.read_csv('../data/weather_v1.csv', index_col = 0)

In [37]:
codesum = codesum.merge(keys, on = 'station_nbr' )

In [38]:
codesum = codesum.loc[:,['date','store_nbr','is_holiday','codesum']]

In [39]:
tmp = tmp.merge(codesum, on = ['date','store_nbr'])

In [40]:
tmp.head()

,date,store_nbr,item_nbr,units,depart,heat,cool,snowfall,preciptotal,resultdir,is_holiday,codesum
0,2012-01-01,1,9,29,10.0,23.0,0.0,0.0,0.05,20.0,holiday,Not_MO
1,2012-01-01,1,28,2,10.0,23.0,0.0,0.0,0.05,20.0,holiday,Not_MO
2,2012-01-01,1,51,1,10.0,23.0,0.0,0.0,0.05,20.0,holiday,Not_MO
3,2012-01-02,1,9,60,23.0,24.0,0.0,0.0,0.01,24.0,holiday_work,MO
4,2012-01-02,1,28,5,23.0,24.0,0.0,0.0,0.01,24.0,holiday_work,MO


## 2-1. codesum에서 holiday 컬럼, codesum 컬럼 category변환 함수 
- holiday 이면 1 아니면 0
- codesum 'Not_MO'이면 1 MO 이면 0 

In [41]:
def cate_holiday(holiday):
    if holiday == 'holiday' or holiday == 'holiday_work':
            return 1
    else:
        return 0
def cate_codesum(codesum):
    if codesum == 'MO':
        return 0 
    else:
        return 1
    
tmp['is_holiday'] = tmp['is_holiday'].apply(cate_holiday)
tmp['codesum'] = tmp['codesum'].apply(cate_codesum)

In [42]:
tmp.head()

,date,store_nbr,item_nbr,units,depart,heat,cool,snowfall,preciptotal,resultdir,is_holiday,codesum
0,2012-01-01,1,9,29,10.0,23.0,0.0,0.0,0.05,20.0,1,1
1,2012-01-01,1,28,2,10.0,23.0,0.0,0.0,0.05,20.0,1,1
2,2012-01-01,1,51,1,10.0,23.0,0.0,0.0,0.05,20.0,1,1
3,2012-01-02,1,9,60,23.0,24.0,0.0,0.0,0.01,24.0,1,0
4,2012-01-02,1,28,5,23.0,24.0,0.0,0.0,0.01,24.0,1,0


# 3. model 생성
- formula = 'units ~ C(store_nbr) + C(item_nbr)+ scale(cool) + scale(heat) + scale(preciptotal) + scale(snowfall) + is_holiday'

In [43]:
len(tmp)

118664

In [44]:
formula = 'units ~ C(store_nbr) + C(item_nbr)+ scale(cool) + scale(heat) + scale(preciptotal) + scale(snowfall) + is_holiday'
model = sm.OLS.from_formula(formula, data= tmp)
result = model.fit()

In [45]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.593
Model:                            OLS   Adj. R-squared:                  0.593
Method:                 Least Squares   F-statistic:                     1088.
Date:                Tue, 13 Mar 2018   Prob (F-statistic):               0.00
Time:                        02:30:23   Log-Likelihood:            -5.7574e+05
No. Observations:              118664   AIC:                         1.152e+06
Df Residuals:                  118504   BIC:                         1.153e+06
Df Model:                         159                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             15.1092      2

## 3-1. cross validation 확인

In [46]:
len(tmp)

118664

In [47]:
X = tmp.loc[:, ['date', 'store_nbr', 'item_nbr', 'cool', 'heat', 'preciptotal','snowfall', 'is_holiday']]
y = tmp.loc[:, ['units']]

In [48]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=4, shuffle=True, random_state=0)
for train_index, test_index in cv.split(X):
    print("test index :", test_index, len(test_index))
    print("." * 80)
    print("train index:", train_index, len(train_index))
    print("=" * 80)

    X_train = X.loc[train_index, :]
    X_test = X.loc[test_index, :]
    y_train = y.loc[train_index, :]
    y_test = y.loc[test_index, :]
    
    tmp_data = pd.concat([X_train, y_train], axis = 1)
    formula = 'units ~ C(store_nbr) + C(item_nbr)+ scale(cool) + scale(heat) + scale(preciptotal) + scale(snowfall) + is_holiday'
    
    model = sm.OLS.from_formula(formula, data= tmp_data)
    result = model.fit()
    
    
    y_hat = result.predict(X_test)
    
    
    
    print(((y_hat - y_hat.mean())**2).sum()  / ((y_test-y_test.mean())**2).sum() )

test index : [     3      6      9 ..., 118659 118660 118663] 29666
................................................................................
train index: [     0      1      2 ..., 118658 118661 118662] 88998
units    0.460212
dtype: float64
test index : [     1      8     11 ..., 118652 118655 118662] 29666
................................................................................
train index: [     0      2      3 ..., 118660 118661 118663] 88998
units    0.687306
dtype: float64
test index : [     0      4      5 ..., 118642 118653 118658] 29666
................................................................................
train index: [     1      2      3 ..., 118661 118662 118663] 88998
units    0.594435
dtype: float64
test index : [     2     10     13 ..., 118641 118649 118661] 29666
................................................................................
train index: [     0      1      3 ..., 118660 118662 118663] 88998
units    0.698335
dtype: float64


# 4. Outlier 제거 코드

In [49]:
len(tmp)

118664

In [203]:
res = result.resid_pearson
index = result.resid.index
outlier_index = abs(res) >= 2

row = pd.Series(index=index, data=res)
drop_index = row[outlier_index].index
len(drop_index)

tmp.drop(list(drop_index),inplace=True)
tmp.reset_index(drop = True, inplace = True)

X = tmp.loc[:, ['date', 'store_nbr', 'item_nbr', 'cool', 'heat', 'preciptotal','snowfall', 'is_holiday']]
y = tmp.loc[:, ['units']]

## 4-1. 2차 제거 후 cross_val 확인
- 위 코드 두번 실행

In [204]:
len(tmp)

86402

In [205]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=4, shuffle=True, random_state=0)
for train_index, test_index in cv.split(X):
    print("test index :", test_index, len(test_index))
    print("." * 80)
    print("train index:", train_index, len(train_index))
    print("=" * 80)

    X_train = X.loc[train_index, :]
    X_test = X.loc[test_index, :]
    y_train = y.loc[train_index, :]
    y_test = y.loc[test_index, :]
    
    tmp_data = pd.concat([X_train, y_train], axis = 1)
    formula = 'units ~ C(store_nbr) + C(item_nbr)+ scale(cool) + scale(heat) + scale(preciptotal) + scale(snowfall) + is_holiday'
    
    model = sm.OLS.from_formula(formula, data= tmp_data)
    result = model.fit()
    
    
    y_hat = result.predict(X_test)
    
    
    
    print(((y_hat - y_hat.mean())**2).sum()  / ((y_test-y_test.mean())**2).sum() )

test index : [    1     3     8 ..., 86394 86400 86401] 21601
................................................................................
train index: [    0     2     4 ..., 86397 86398 86399] 64801
units    0.923689
dtype: float64
test index : [    0     5     6 ..., 86395 86396 86397] 21601
................................................................................
train index: [    1     2     3 ..., 86399 86400 86401] 64801
units    0.922208
dtype: float64
test index : [    4     7    15 ..., 86391 86393 86398] 21600
................................................................................
train index: [    0     1     2 ..., 86399 86400 86401] 64802
units    0.92653
dtype: float64
test index : [    2    10    13 ..., 86383 86384 86399] 21600
................................................................................
train index: [    0     1     3 ..., 86398 86400 86401] 64802
units    0.921539
dtype: float64


# 4-2. 현재모델

In [206]:
formula = 'units ~ C(store_nbr) + C(item_nbr)+ scale(cool) + scale(heat) + scale(preciptotal) + scale(snowfall) + is_holiday'
model = sm.OLS.from_formula(formula, data= tmp)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.924
Model:                            OLS   Adj. R-squared:                  0.923
Method:                 Least Squares   F-statistic:                     6550.
Date:                Tue, 13 Mar 2018   Prob (F-statistic):               0.00
Time:                        02:54:59   Log-Likelihood:            -2.9097e+05
No. Observations:               86402   AIC:                         5.823e+05
Df Residuals:                   86242   BIC:                         5.838e+05
Df Model:                         159                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              8.8841      0

In [207]:
# 정규화  result1 = Ridge 모형
result1 = model.fit_regularized(alpha=0.1, L1_wt=0)

# 정규화  result2 = Lasso 모형
result2 = model.fit_regularized(alpha=0.1, L1_wt=1)

# 정규화  result3 = Elastic Net 모형
result3 = model.fit_regularized(alpha=0.1, L1_wt=0.5)

# 5. test.csv load

In [208]:
test = pd.read_csv('../data/basic/test.csv', index_col = 0)

In [209]:
test.reset_index(inplace = True)

In [210]:
test.tail()

,date,store_nbr,item_nbr
526912,2014-10-26,45,107
526913,2014-10-26,45,108
526914,2014-10-26,45,109
526915,2014-10-26,45,110
526916,2014-10-26,45,111


## 5-1. test 데이터에 weather's feature merge후 test_df로 저장

In [211]:
test_df = test.merge(keys).merge(weather)

In [212]:
test_df.tail()

,date,store_nbr,item_nbr,station_nbr,tmax,tmin,tavg,depart,dewpoint,wetbulb,...,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day,codesum
526912,2014-08-02,40,107,13,81.0,51.0,66.0,3.0,47.0,56.0,...,0.0,24.44,30.12,5.7,17.0,6.5,2014,8,2,
526913,2014-08-02,40,108,13,81.0,51.0,66.0,3.0,47.0,56.0,...,0.0,24.44,30.12,5.7,17.0,6.5,2014,8,2,
526914,2014-08-02,40,109,13,81.0,51.0,66.0,3.0,47.0,56.0,...,0.0,24.44,30.12,5.7,17.0,6.5,2014,8,2,
526915,2014-08-02,40,110,13,81.0,51.0,66.0,3.0,47.0,56.0,...,0.0,24.44,30.12,5.7,17.0,6.5,2014,8,2,
526916,2014-08-02,40,111,13,81.0,51.0,66.0,3.0,47.0,56.0,...,0.0,24.44,30.12,5.7,17.0,6.5,2014,8,2,


In [213]:
etc = ['day','month','year', 'tavg', 'sealevel', 'wetbulb', 'stnpressure', 'tmin', 'tmax', 'dewpoint', 'avgspeed', 'resultspeed']
category = ['sunrise','sunset','codesum']

for each in etc+category:
    test_df.drop(each,axis=1,inplace=True)

In [214]:
test_df.tail()

,date,store_nbr,item_nbr,station_nbr,depart,heat,cool,snowfall,preciptotal,resultdir
526912,2014-08-02,40,107,13,3.0,0.0,1.0,0.0,0.0,17.0
526913,2014-08-02,40,108,13,3.0,0.0,1.0,0.0,0.0,17.0
526914,2014-08-02,40,109,13,3.0,0.0,1.0,0.0,0.0,17.0
526915,2014-08-02,40,110,13,3.0,0.0,1.0,0.0,0.0,17.0
526916,2014-08-02,40,111,13,3.0,0.0,1.0,0.0,0.0,17.0


In [215]:
test_df = test_df.merge(codesum, on = ['date','store_nbr'])
test_df['is_holiday'] = test_df['is_holiday'].apply(cate_holiday)
test_df['codesum'] = test_df['codesum'].apply(cate_codesum)

## 5-2. test_df에서 팔린 units의 feature에 대해서만 선택후 test_X_df에 저장

In [216]:
test_X_df = pd.DataFrame(columns=['date', 'store_nbr', 'item_nbr', 'station_nbr', 'cool', 'depart', 'heat', 'preciptotal', 'resultdir', 'snowfall', 'is_holiday', 'codesum'])

for nbr in range(1, 45+1):
    store_nbr = tmp[tmp['store_nbr'].isin([nbr])]
    sold_item_list = list(store_nbr['item_nbr'].unique())
    sold_item_list

    test_store_nbr = test_df[test_df['store_nbr'].isin([nbr])]
    to_test_df = test_store_nbr[test_store_nbr['item_nbr'].isin(sold_item_list)]

    test_X_df = pd.concat([test_X_df, to_test_df], axis = 0)
    
test_X_df.shape

(25267, 12)

In [217]:
test_X_df.head()

,codesum,cool,date,depart,heat,is_holiday,item_nbr,preciptotal,resultdir,snowfall,station_nbr,store_nbr
484301,0,0.0,2013-06-04,9.0,3.0,0,9,0.0,31.0,0.0,1,1
484320,0,0.0,2013-06-04,9.0,3.0,0,28,0.0,31.0,0.0,1,1
484332,0,0.0,2013-06-04,9.0,3.0,0,40,0.0,31.0,0.0,1,1
484339,0,0.0,2013-06-04,9.0,3.0,0,47,0.0,31.0,0.0,1,1
484343,0,0.0,2013-06-04,9.0,3.0,0,51,0.0,31.0,0.0,1,1


In [218]:
test_X_df.head()

,codesum,cool,date,depart,heat,is_holiday,item_nbr,preciptotal,resultdir,snowfall,station_nbr,store_nbr
484301,0,0.0,2013-06-04,9.0,3.0,0,9,0.0,31.0,0.0,1,1
484320,0,0.0,2013-06-04,9.0,3.0,0,28,0.0,31.0,0.0,1,1
484332,0,0.0,2013-06-04,9.0,3.0,0,40,0.0,31.0,0.0,1,1
484339,0,0.0,2013-06-04,9.0,3.0,0,47,0.0,31.0,0.0,1,1
484343,0,0.0,2013-06-04,9.0,3.0,0,51,0.0,31.0,0.0,1,1


## 5-3. test_df와 type을 맞춰줘야함 그래야 predict()에 넣을 수 있다.

In [219]:
test_X_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25267 entries, 484301 to 514074
Data columns (total 12 columns):
codesum        25267 non-null object
cool           25267 non-null float64
date           25267 non-null object
depart         25267 non-null float64
heat           25267 non-null float64
is_holiday     25267 non-null object
item_nbr       25267 non-null object
preciptotal    25267 non-null float64
resultdir      25267 non-null float64
snowfall       25267 non-null float64
station_nbr    25267 non-null object
store_nbr      25267 non-null object
dtypes: float64(6), object(6)
memory usage: 2.5+ MB


In [220]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526917 entries, 0 to 526916
Data columns (total 12 columns):
date           526917 non-null object
store_nbr      526917 non-null int64
item_nbr       526917 non-null int64
station_nbr    526917 non-null int64
depart         526917 non-null float64
heat           526917 non-null float64
cool           526917 non-null float64
snowfall       526917 non-null float64
preciptotal    526917 non-null float64
resultdir      526917 non-null float64
is_holiday     526917 non-null int64
codesum        526917 non-null int64
dtypes: float64(6), int64(5), object(1)
memory usage: 52.3+ MB


In [221]:
test_X_df['store_nbr'] = test_X_df['store_nbr'].astype(int)
test_X_df['item_nbr'] = test_X_df['item_nbr'].astype(int)
test_X_df['station_nbr'] = test_X_df['station_nbr'].astype(int)
test_X_df['is_holiday'] = test_X_df['is_holiday'].astype(int)
test_X_df['codesum'] = test_X_df['codesum'].astype(int)

## 5-4. 이제 test_X_df 를 predict() 하면

### ridge모형
- result1

In [222]:
y_hat1 = result1.predict(test_X_df)

In [223]:
y_df1 = test_df.loc[:,['date','store_nbr','item_nbr']]

In [224]:
y_df1['units'] = y_hat1

In [225]:
y_df1.tail()

,date,store_nbr,item_nbr,units
526912,2014-08-02,40,107,NaN
526913,2014-08-02,40,108,NaN
526914,2014-08-02,40,109,NaN
526915,2014-08-02,40,110,NaN
526916,2014-08-02,40,111,NaN


In [226]:
def itiszero(units):
    if units > 0:
        return units
    else:
        return 0

In [227]:
y_df1['units'] = y_df1['units'].apply(itiszero)
y_df1['units'] = y_df1['units'].astype(int)

In [228]:
y_df1.head()

,date,store_nbr,item_nbr,units
0,2013-04-01,2,1,0
1,2013-04-01,2,2,0
2,2013-04-01,2,3,0
3,2013-04-01,2,4,0
4,2013-04-01,2,5,27


### Lasso 모형
- result2

In [229]:
y_hat2 = result2.predict(test_X_df)
y_df2 = test_df.loc[:,['date','store_nbr','item_nbr']]
y_df2['units'] = y_hat2
y_df2.tail()

y_df2['units'] = y_df2['units'].apply(itiszero)
y_df2['units'] = y_df2['units'].astype(int)
y_df2.head()

,date,store_nbr,item_nbr,units
0,2013-04-01,2,1,0
1,2013-04-01,2,2,0
2,2013-04-01,2,3,0
3,2013-04-01,2,4,0
4,2013-04-01,2,5,35


### Elastic Net 모형
- result3

In [230]:
y_hat3 = result3.predict(test_X_df)
y_df3 = test_df.loc[:,['date','store_nbr','item_nbr']]
y_df3['units'] = y_hat3
y_df3.tail()

y_df3['units'] = y_df3['units'].apply(itiszero)
y_df3['units'] = y_df3['units'].astype(int)
y_df3.head()

,date,store_nbr,item_nbr,units
0,2013-04-01,2,1,0
1,2013-04-01,2,2,0
2,2013-04-01,2,3,0
3,2013-04-01,2,4,0
4,2013-04-01,2,5,31


In [231]:
y_hat = result.predict(test_X_df)

In [232]:
y_df = test_df.loc[:,['date','store_nbr','item_nbr']]

In [233]:
y_df['units'] = y_hat

In [234]:
y_df.tail()

,date,store_nbr,item_nbr,units
526912,2014-08-02,40,107,NaN
526913,2014-08-02,40,108,NaN
526914,2014-08-02,40,109,NaN
526915,2014-08-02,40,110,NaN
526916,2014-08-02,40,111,NaN


In [235]:
def itiszero(units):
    if units > 0:
        return units
    else:
        return 0

In [236]:
y_df['units'] = y_df['units'].apply(itiszero)

In [237]:
y_df['units'] = y_df['units'].astype(int)

In [238]:
y_df

,date,store_nbr,item_nbr,units
0,2013-04-01,2,1,0
1,2013-04-01,2,2,0
2,2013-04-01,2,3,0
3,2013-04-01,2,4,0
4,2013-04-01,2,5,50
5,2013-04-01,2,6,0
6,2013-04-01,2,7,0
7,2013-04-01,2,8,0
8,2013-04-01,2,9,0
9,2013-04-01,2,10,0


# 6. 제출 형식에 맞게 수정

### 정규화 X

In [251]:
y_df['id'] = y_df['store_nbr'].astype(str)+"_"+y_df['item_nbr'].astype(str)+'_'+y_df['date'].astype(str)

In [253]:
for_submission = y_df.loc[:,['id','units']]

In [254]:
for_submission.head()

,id,units
0,2_1_2013-04-01,0
1,2_2_2013-04-01,0
2,2_3_2013-04-01,0
3,2_4_2013-04-01,0
4,2_5_2013-04-01,50


In [255]:
for_submission.to_csv('../model_generation/submission_csv/submission.csv', index = False)

### result1

In [239]:
y_df1['id'] = y_df1['store_nbr'].astype(str)+"_"+y_df1['item_nbr'].astype(str)+'_'+y_df1['date'].astype(str)

In [240]:
for_submission1 = y_df1.loc[:,['id','units']]

In [241]:
for_submission1.head()

,id,units
0,2_1_2013-04-01,0
1,2_2_2013-04-01,0
2,2_3_2013-04-01,0
3,2_4_2013-04-01,0
4,2_5_2013-04-01,27


In [242]:
for_submission1.to_csv('../model_generation/submission_csv/submission1.csv', index = False)

### result2

In [243]:
y_df2['id'] = y_df2['store_nbr'].astype(str)+"_"+y_df2['item_nbr'].astype(str)+'_'+y_df2['date'].astype(str)

In [244]:
for_submission2 = y_df2.loc[:,['id','units']]

In [245]:
for_submission2.head()

,id,units
0,2_1_2013-04-01,0
1,2_2_2013-04-01,0
2,2_3_2013-04-01,0
3,2_4_2013-04-01,0
4,2_5_2013-04-01,35


In [246]:
for_submission2.to_csv('../model_generation/submission_csv/submission2.csv', index = False)

### result3

In [247]:
y_df3['id'] = y_df3['store_nbr'].astype(str)+"_"+y_df3['item_nbr'].astype(str)+'_'+y_df3['date'].astype(str)

In [248]:
for_submission3 = y_df3.loc[:,['id','units']]

In [249]:
for_submission3.head()

,id,units
0,2_1_2013-04-01,0
1,2_2_2013-04-01,0
2,2_3_2013-04-01,0
3,2_4_2013-04-01,0
4,2_5_2013-04-01,31


In [250]:
for_submission3.to_csv('../model_generation/submission_csv/submission3.csv', index = False)